Task 3
---------
Scrap the books (name, price, rate) for each category and put them into a CSV & Excel file
https://books.toscrape.com/


In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

url = "https://books.toscrape.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all categories
categories = soup.find('div', {'class': 'side_categories'}).find_all('li')

with open('books.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Category", "Book Name", "Price", "Rating"])

# Create an Excel file with a context manager
with pd.ExcelWriter('books.xlsx', engine='openpyxl') as excel_file:
    for category in categories:
        category_url = category.find('a')['href']
        category_name = category.find('a').text.strip()
        
        full_category_url = url + category_url
        response = requests.get(full_category_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all books in the category
        books = soup.find_all('article', {'class': 'product_pod'})
        
        book_data = []  # Create a list to store the book data
        
        for book in books:
            book_name = book.find('h3').find('a')['title']
            price = book.find('p', {'class': 'price_color'}).text.strip()
            rating = book.find('p', {'class': 'star-rating'}).get('class')[1]
            
            with open('books.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([category_name, book_name, price, rating])

            book_data.append([category_name, book_name, price, rating])
        
        # Write the data to the Excel file
        df = pd.DataFrame(book_data, columns=["Category", "Book Name", "Price", "Rating"])
        df.to_excel(excel_file, sheet_name=category_name, index=False)
